# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 14 2022 3:37PM,253138,19,60020318,"GCH Granules USA, Inc.",Released
1,Dec 14 2022 3:35PM,253137,19,ADV80009123,"AdvaGen Pharma, Ltd",Released
2,Dec 14 2022 3:35PM,253137,19,ADV80009124,"AdvaGen Pharma, Ltd",Released
3,Dec 14 2022 3:35PM,253137,19,ADV80009125,"AdvaGen Pharma, Ltd",Released
4,Dec 14 2022 3:35PM,253137,19,ADV80009126,"AdvaGen Pharma, Ltd",Released
5,Dec 14 2022 3:35PM,253137,19,ADV80009127,"AdvaGen Pharma, Ltd",Released
6,Dec 14 2022 3:35PM,253137,19,ADV80009128,"AdvaGen Pharma, Ltd",Released
7,Dec 14 2022 3:35PM,253137,19,ADV80009133,"AdvaGen Pharma, Ltd",Released
8,Dec 14 2022 3:32PM,253136,19,8902916,"Emersa Waterbox, LLC",Released
9,Dec 14 2022 3:23PM,253135,20,8898667,Else Nutrition,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,253134,Released,1
36,253135,Released,1
37,253136,Released,1
38,253137,Released,7
39,253138,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253134,NaN,NaN,1.0
253135,NaN,NaN,1.0
253136,NaN,NaN,1.0
253137,NaN,NaN,7.0
253138,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253051,5.0,6.0,4.0
253053,14.0,7.0,4.0
253064,0.0,0.0,23.0
253083,0.0,0.0,13.0
253086,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253051,5,6,4
253053,14,7,4
253064,0,0,23
253083,0,0,13
253086,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253051,5,6,4
1,253053,14,7,4
2,253064,0,0,23
3,253083,0,0,13
4,253086,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253051,5,6,4
1,253053,14,7,4
2,253064,,,23
3,253083,,,13
4,253086,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 14 2022 3:37PM,253138,19,"GCH Granules USA, Inc."
1,Dec 14 2022 3:35PM,253137,19,"AdvaGen Pharma, Ltd"
8,Dec 14 2022 3:32PM,253136,19,"Emersa Waterbox, LLC"
9,Dec 14 2022 3:23PM,253135,20,Else Nutrition
10,Dec 14 2022 3:06PM,253134,10,"Senseonics, Incorporated"
11,Dec 14 2022 2:48PM,253132,21,"NBTY Global, Inc."
12,Dec 14 2022 2:48PM,253131,21,"NBTY Global, Inc."
13,Dec 14 2022 2:43PM,253129,10,"Senseonics, Incorporated"
14,Dec 14 2022 2:38PM,253128,10,"Snap Medical Industries, LLC"
16,Dec 14 2022 2:34PM,253127,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 14 2022 3:37PM,253138,19,"GCH Granules USA, Inc.",,,1
1,Dec 14 2022 3:35PM,253137,19,"AdvaGen Pharma, Ltd",,,7
2,Dec 14 2022 3:32PM,253136,19,"Emersa Waterbox, LLC",,,1
3,Dec 14 2022 3:23PM,253135,20,Else Nutrition,,,1
4,Dec 14 2022 3:06PM,253134,10,"Senseonics, Incorporated",,,1
5,Dec 14 2022 2:48PM,253132,21,"NBTY Global, Inc.",,,1
6,Dec 14 2022 2:48PM,253131,21,"NBTY Global, Inc.",,1,
7,Dec 14 2022 2:43PM,253129,10,"Senseonics, Incorporated",,1,
8,Dec 14 2022 2:38PM,253128,10,"Snap Medical Industries, LLC",,,2
9,Dec 14 2022 2:34PM,253127,19,"AdvaGen Pharma, Ltd",,,16


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 14 2022 3:37PM,253138,19,"GCH Granules USA, Inc.",1,,
1,Dec 14 2022 3:35PM,253137,19,"AdvaGen Pharma, Ltd",7,,
2,Dec 14 2022 3:32PM,253136,19,"Emersa Waterbox, LLC",1,,
3,Dec 14 2022 3:23PM,253135,20,Else Nutrition,1,,
4,Dec 14 2022 3:06PM,253134,10,"Senseonics, Incorporated",1,,
5,Dec 14 2022 2:48PM,253132,21,"NBTY Global, Inc.",1,,
6,Dec 14 2022 2:48PM,253131,21,"NBTY Global, Inc.",,1,
7,Dec 14 2022 2:43PM,253129,10,"Senseonics, Incorporated",,1,
8,Dec 14 2022 2:38PM,253128,10,"Snap Medical Industries, LLC",2,,
9,Dec 14 2022 2:34PM,253127,19,"AdvaGen Pharma, Ltd",16,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 14 2022 3:37PM,253138,19,"GCH Granules USA, Inc.",1,,
1,Dec 14 2022 3:35PM,253137,19,"AdvaGen Pharma, Ltd",7,,
2,Dec 14 2022 3:32PM,253136,19,"Emersa Waterbox, LLC",1,,
3,Dec 14 2022 3:23PM,253135,20,Else Nutrition,1,,
4,Dec 14 2022 3:06PM,253134,10,"Senseonics, Incorporated",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 14 2022 3:37PM,253138,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
1,Dec 14 2022 3:35PM,253137,19,"AdvaGen Pharma, Ltd",7.0,NaN,NaN
2,Dec 14 2022 3:32PM,253136,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
3,Dec 14 2022 3:23PM,253135,20,Else Nutrition,1.0,NaN,NaN
4,Dec 14 2022 3:06PM,253134,10,"Senseonics, Incorporated",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 14 2022 3:37PM,253138,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
1,Dec 14 2022 3:35PM,253137,19,"AdvaGen Pharma, Ltd",7.0,0.0,0.0
2,Dec 14 2022 3:32PM,253136,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
3,Dec 14 2022 3:23PM,253135,20,Else Nutrition,1.0,0.0,0.0
4,Dec 14 2022 3:06PM,253134,10,"Senseonics, Incorporated",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3037284,70.0,4.0,10.0
15,1518509,55.0,13.0,19.0
16,253125,1.0,0.0,0.0
19,2531154,34.0,0.0,0.0
20,759340,3.0,0.0,0.0
21,506263,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3037284,70.0,4.0,10.0
1,15,1518509,55.0,13.0,19.0
2,16,253125,1.0,0.0,0.0
3,19,2531154,34.0,0.0,0.0
4,20,759340,3.0,0.0,0.0
5,21,506263,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,70.0,4.0,10.0
1,15,55.0,13.0,19.0
2,16,1.0,0.0,0.0
3,19,34.0,0.0,0.0
4,20,3.0,0.0,0.0
5,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,70.0
1,15,Released,55.0
2,16,Released,1.0
3,19,Released,34.0
4,20,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,10.0,19.0,0.0,0.0,0.0,0.0
Executing,4.0,13.0,0.0,0.0,0.0,1.0
Released,70.0,55.0,1.0,34.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,10.0,19.0,0.0,0.0,0.0,0.0
1,Executing,4.0,13.0,0.0,0.0,0.0,1.0
2,Released,70.0,55.0,1.0,34.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,10.0,19.0,0.0,0.0,0.0,0.0
1,Executing,4.0,13.0,0.0,0.0,0.0,1.0
2,Released,70.0,55.0,1.0,34.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()